In [11]:
import pandas as pd

In [32]:
chicago_flu = pd.read_csv("tracts.csv").drop(columns=["week", "pop", "illit.r", "den.r"])
deaths_total = chicago_flu.groupby("gisjoin").counts.sum()
chicago_flu = chicago_flu[~chicago_flu.duplicated("gisjoin")].set_index("gisjoin").drop(columns="counts")
chicago_flu["death_count"] = deaths_total
chicago_flu = chicago_flu.rename(columns={c:c.replace(".", "_").lower() for c in chicago_flu.columns})
chicago_flu.index.name = "geography_code"

In [34]:
chicago_flu.to_csv("../../chapter_02/data/chicago_influenza_1918.csv")